In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense , Dropout
import os

/Users/ankitprasad/miniforge3/envs/tensorflow/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
sz = 128

In [3]:
classifier = Sequential()

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-04-08 21:08:54.456486: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-08 21:08:54.456672: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
# First convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), input_shape=(sz, sz, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), activation='relu'))

classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=96, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dense(units=26, activation='softmax')) 

classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # categorical_crossentropy for more than 2


classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 28800)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3686528   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0

In [5]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('/Users/ankitprasad/Desktop/Project Hand/data/train',
                                                 target_size=(sz, sz),
                                                 batch_size=10,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('/Users/ankitprasad/Desktop/Project Hand/data/test',
                                            target_size=(sz , sz),
                                            batch_size=10,
                                            color_mode='grayscale',
                                            class_mode='categorical') 

Found 11712 images belonging to 26 classes.
Found 3979 images belonging to 26 classes.


In [6]:
print(len(training_set))
print(len(test_set))

1172
398


In [7]:
classifier.fit_generator(
        training_set,
        steps_per_epoch=len(training_set), # No of images in training set
        epochs=5,
        validation_data=test_set,
        validation_steps=len(test_set))# No of images in test set

/Users/ankitprasad/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2023-04-08 21:09:02.152517: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-04-08 21:09:02.155045: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/5


2023-04-08 21:09:08.315379: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1171/1172 [============================>.] - ETA: 0s - loss: 2.7278 - accuracy: 0.1857

2023-04-08 21:09:33.301726: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1172/1172 [==============================] - 35s 24ms/step - loss: 2.7266 - accuracy: 0.1860 - val_loss: 1.1518 - val_accuracy: 0.6042
Epoch 2/5
1172/1172 [==============================] - 29s 24ms/step - loss: 1.0133 - accuracy: 0.6536 - val_loss: 0.7327 - val_accuracy: 0.7514
Epoch 3/5
1172/1172 [==============================] - 28s 24ms/step - loss: 0.6631 - accuracy: 0.7712 - val_loss: 0.6397 - val_accuracy: 0.7723
Epoch 4/5
1172/1172 [==============================] - 28s 24ms/step - loss: 0.5152 - accuracy: 0.8240 - val_loss: 0.7202 - val_accuracy: 0.7540
Epoch 5/5
1172/1172 [==============================] - 28s 24ms/step - loss: 0.4062 - accuracy: 0.8659 - val_loss: 0.5905 - val_accuracy: 0.7987


In [8]:
# Saving the model
model_json = classifier.to_json()
with open("modelMain.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('modelMain.h5')
print('Weights saved')

Model Saved
Weights saved
